# MNIST Deep Convolutional Generative Adversarial Networks
Implementation of DCGAN on MNIST dataset.

#### Losses
* Discriminator Loss
* Generator Loss

The discriminator will induce the generator to produce samples similar to the real samples.

#### References
* [Paper](https://arxiv.org/pdf/1406.2661.pdf)
* [Code](https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/dcgan/dcgan.py)
* [Various GAN Implementation on Pytorch](https://github.com/eriklindernoren/PyTorch-GAN)
* [DCGAN Implementation](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)
* [GAN Hacks](https://github.com/soumith/ganhacks)
* [Clone and Detach](https://discuss.pytorch.org/t/clone-and-detach-in-v0-4-0/16861/2)
* [Training GANs](https://www.youtube.com/watch?v=X1mUN6dD8uE)

In [1]:
import mnist_data_pytorch as data
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from tqdm import tqdm
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)
print('Pytorch version:', torch.__version__)
# Tensorboard
from torch.utils.tensorboard import SummaryWriter
!rm -rf ./runs
writer = SummaryWriter('./runs/train')

# Metaparameters
num_epochs = 100
num_classes = 10
latent_size = 64
gen_lr = 0.0002
disc_lr = 0.0002
EPS = 1e-15

Device: cuda:0
Pytorch version: 1.2.0


#### Define Encoder/Decoder/Discriminator

In [2]:
class Generator(nn.Module):
    def __init__(self, latent_size=100, img_size=28, channels=1):
        super(Generator, self).__init__()
        
        # Used to make z_sample larger to fit a 2d activation map
        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_size, 128 * self.init_size ** 2))

        self.convs = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            #nn.Tanh()
            nn.Sigmoid() #Sigmoid bit better for MNIST
        )

    def forward(self, z):
        batch_size = z.shape[0]
        x = self.l1(z)
        # Reshape activation to 2d map
        x = x.view(batch_size, 128, self.init_size, self.init_size)
        x = self.convs(x)
        return x


class Discriminator(nn.Module):
    def __init__(self, channels=1, img_size=28):
        super(Discriminator, self).__init__()
        
        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )
        
        self.adv_layer = nn.Sequential(nn.Linear(512, 1), nn.Sigmoid())

    def forward(self, x):
        batch_size = x.shape[0]
        x = self.model(x)
        x = x.view(batch_size, -1)
        real_or_fake = self.adv_layer(x)
        return real_or_fake

# Initialize Networks
G = Generator(latent_size=latent_size).to(device)
D = Discriminator().to(device)

#### Initialize Optimizers

In [3]:
optim_generator = torch.optim.Adam(G.parameters(), lr=gen_lr)
optim_discriminator = torch.optim.Adam(D.parameters(), lr=disc_lr)

#### Train loop

In [ ]:
k = 1
for epoch in tqdm(range(num_epochs)):
    running_loss_G = 0.0
    running_loss_D = 0.0
    # Iterate over the data
    for idx_sample, (real_imgs, _) in enumerate(data.dataloaders['train']):
        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size()[0]
        
        # ---------------------
        #  Train Discriminator
        # ---------------------
        for _ in range(k):
            # Adversarial ground truths (you can do soft-label here....)
            valid = (torch.ones(batch_size, 1).fill_(1.0)).to(device)
            fake = (torch.ones(batch_size, 1).fill_(0.0)).to(device)
            z_sample = torch.randn(batch_size, latent_size).to(device)

            gen_samples = G(z_sample)
            optim_discriminator.zero_grad()
            # Measure discriminator's ability to classify real from generated samples
            real_loss = F.binary_cross_entropy(D(real_imgs), valid)
            fake_loss = F.binary_cross_entropy(D(gen_samples.detach()), fake)
            d_loss = (real_loss + fake_loss) / 2

            d_loss.backward()
            optim_discriminator.step()
        
        # ---------------------
        #  Train Generator
        # ---------------------
        optim_generator.zero_grad()
        # Sample from distribution Z (z~Z)
        z_sample = torch.randn(batch_size, latent_size).to(device)

        # Loss measures generator's ability to fool the discriminator
        gen_samples = G(z_sample)
        g_loss = F.binary_cross_entropy(D(gen_samples), valid)

        g_loss.backward()
        optim_generator.step()
        
        # Update statistics
        running_loss_G += g_loss.item() * batch_size
        # Update statistics
        running_loss_D += d_loss.item() * batch_size
    
    # Epoch ends
    epoch_loss_generator = running_loss_G / len(data.dataloaders['train'].dataset)
    epoch_loss_discriminator = running_loss_D / len(data.dataloaders['train'].dataset)
    
    # Send results to tensorboard
    writer.add_scalar('train/loss_generator', epoch_loss_generator, epoch)
    writer.add_scalar('train/loss_discriminator', epoch_loss_discriminator, epoch)
    
    # Send images to tensorboard
    writer.add_images('train/gen_samples', gen_samples.view(batch_size,1,28,28), epoch)
    writer.add_images('train/input_images', real_imgs.view(batch_size,1,28,28), epoch)
    
    # Send latent to tensorboard
    writer.add_histogram('train/latent', z_sample, epoch)
    writer.add_histogram('train/X', real_imgs, epoch)
    writer.add_histogram('train/G(z)', gen_samples, epoch)

 79%|███████▉  | 79/100 [20:29<05:24, 15.47s/it]

#### Generate Samples (Unconditioned)
Observe that the generated samples are somehow a mix of all classes.

In [ ]:
def generate_sample(num_idx=0):
    G.eval()
    
    z = torch.randn(1, latent_size).to(device)
    with torch.no_grad(): 
        generated_sample = G(z)

    plt.imshow(generated_sample.view(28,28).cpu().numpy())
    plt.title('Generated sample')
    plt.show()

In [ ]:
interact(generate_sample, num_idx=widgets.IntSlider(min=0, max=100, step=1, value=0));